In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 10:16:58,663 INFO: Initializing external client
2025-03-04 10:16:58,664 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 10:16:59,813 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681
Fetching data from 2025-02-03 15:16:58.661611+00:00 to 2025-03-04 14:16:58.661611+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.38s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 16:00:00+00:00,2,0
1,2025-02-03 17:00:00+00:00,2,0
2,2025-02-03 18:00:00+00:00,2,0
3,2025-02-03 19:00:00+00:00,2,0
4,2025-02-03 20:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 10:00:00+00:00,263,116
174441,2025-03-04 11:00:00+00:00,263,98
174442,2025-03-04 12:00:00+00:00,263,100
174443,2025-03-04 13:00:00+00:00,263,100


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 1 to 177205
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 1 to 177205
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,99,65,6,168,44,30,30,24,15,3,...,20,57,3,44,35,12,93,84,114,2025-02-07 20:00:00
1,0,2,0,0,0,0,0,0,0,2,...,2,0,3,0,0,0,0,0,165,2025-02-24 15:00:00
2,1,0,3,1,4,0,1,0,2,0,...,0,2,0,0,0,0,1,0,93,2025-02-09 23:00:00
3,112,11,127,41,81,35,48,104,95,88,...,106,85,25,87,16,17,30,13,263,2025-02-23 17:00:00
4,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,241,2025-02-07 21:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,7,8,0,0,2,1,1,3,1,1,...,7,6,2,3,3,5,3,15,4,2025-02-22 07:00:00
247,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,21,2025-02-10 04:00:00
248,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,94,2025-02-21 06:00:00
249,0,0,0,0,0,1,0,0,0,2,...,0,0,0,0,0,0,1,0,185,2025-02-15 06:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 10:17:18,744 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 10:17:18,752 INFO: Initializing external client
2025-03-04 10:17:18,752 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 10:17:19,800 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681
Fetching data from 2025-02-03 15:17:18.744014+00:00 to 2025-03-04 14:17:18.744014+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.70s) 


In [12]:
current_date

Timestamp('2025-03-04 15:17:18.744014+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 16:00:00
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,3,2025-03-03 16:00:00
2,1,4,0,1,3,3,6,2,3,6,...,11,4,3,5,1,2,2,3,4,2025-03-03 16:00:00
3,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,6,2025-03-03 16:00:00
4,1,1,1,5,1,2,0,0,0,0,...,5,3,3,2,8,3,4,1,7,2025-03-03 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,1,0,0,0,259,2025-03-03 16:00:00
247,3,1,1,3,0,1,0,0,0,0,...,3,0,0,1,4,3,2,5,260,2025-03-03 16:00:00
248,38,30,41,24,17,16,11,3,1,3,...,9,12,7,16,25,36,32,26,261,2025-03-03 16:00:00
249,81,98,73,43,34,22,4,6,1,3,...,172,124,118,98,79,71,82,97,262,2025-03-03 16:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 10:17:33,830 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 10:17:33,838 INFO: Initializing external client
2025-03-04 10:17:33,839 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 10:17:34,903 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,-0.0
1,3,-0.0
2,4,2.0
3,6,0.0
4,7,2.0
...,...,...
246,259,0.0
247,260,1.0
248,261,28.0
249,262,35.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 239, 186, 237,  48, 230,  68, 229,  79, 100])